In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud_module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
host = 'nv-desktop-services.apporto.com'
port = 31275
db = 'AAC'
collection = 'animals'

shelter = AnimalShelter(
    user=username,
    password=password,
    host=host,
    port=port,
    db=db,
    collection=collection
)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.A(
        html.Img(src=f'data:image/png;base64,{encoded_image}',style={'height': '50px'}),
        href='http://learn.snhu.edu/content/enforced/1644165-CS-340-11553.202456-1/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=ScQoRYLN90TGv4T9RbCyYwie6&ou=1332057&ou=1644165',
        target='_blank'
    ),
    html.Br(),
    html.Div('Created by Maxim Colburn', style={'font-size': '16px', 'font-weight': 'bold'}),
    html.Hr(),
    
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
        ],
        value='all'
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable="single",
        sort_action="native",
        filter_action="native",
        selected_rows=[0]
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display' : 'flex', 'justify-content': 'space-between'}, children=[
        html.Div(id='graph-id', style={'flex': '1', 'margin-right': '10px'}),
        html.Div(id='map-id', style={'flex': '1', 'margin-left': '10px'})
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query ={}
    if filter_type != 'all':
        if filter_type == 'water':
            query = {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}}
        elif filter_type =='mountain':
             query = {'breed': {'$in': ['German Shepard', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}}
        elif filter_type == 'disaster':
             query = {'breed': {'$in': ['Doberman Pinscher', 'German Shepard', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}}
    filtered_df = pd.DataFrame.from_records(shelter.read(query))
    filtered_df.drop(columns=['_id'], inplace=True)
    return filtered_df.to_dict('records')
    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if selected_rows is None:
        return []
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or index is None:
        return dl.map(style={'width': '1000px', 'height': '500px'},center=[30.75, -97.48], zoom=10)
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[30.75, -97.48], zoom=10, children=[
                  dl.TileLayer(id="base-layer-id"),
                  dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                           children=[
                               dl.Tooltip(dff.iloc[row, 4]),
                               dl.Popup([
                                   html.H1("Animal Name"),
                                   html.P(dff.iloc[row, 9])
                               ])
                           ])
              ])
    ]
            
        
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
        if viewData is None:
            return
        dff = pd.DataFrame.from_dict(viewData)
        fig = px.pie(dff, names='breed', title='Preferred Breeds')
        return [dcc.Graph(figure=fig)]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:13660/
